In [109]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import time
import pickle

tf.test.is_built_with_cuda()

True

In [110]:
# hyperparameter

max_words = 10000
sentence_len = 20
pred_len = 1  # fix
train_len = sentence_len - pred_len

max_samples = int(6e6) #6e6 max

In [111]:
# pre trained word embeddings

import bcolz

glove_path = './'
vectors = bcolz.open(f'{glove_path}/6B.50.dat')[:]
words = pickle.load(open(f'{glove_path}/6B.50_words.pkl', 'rb'))
word2idx = pickle.load(open(f'{glove_path}/6B.50_idx.pkl', 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}

embedding_dim = 50

# if files do not exist run glove_embedding.ipynb first

In [112]:

def build_vocab(words, max_words=None):
    if max_words is None:
        return list(set(words))
    
    vocab = {}
    for word in word_seq:
        try:
            vocab[word] += 1
        except KeyError:
            vocab[word] = 1
    most_freq = sorted(vocab.keys(), key=vocab.get, reverse=True)
    return most_freq[:max_words]


def replace_by_tbl(text, tbl):
    for k,v in tbl.items():
        text = text.replace(k,v)
    return text


# preprossing mapping tables
apostrophe_tbl = {ord(key): '\'' for key in '\`\´\’'}
shortform_tbl = {
    'n\'t' : ' not',
    '\'ve' : ' have',
    '\'ll' : ' will',
    '\'m' : ' am',
    '\'re' : ' are',
    '\'s' : ' is',
    '\'d' : ' would',
}
remove_apostrophe_tbl = {ord('\''): None}
restore_oclock = {'oclock': 'o\'clock'}


def preprocess_text(text):
    text = text.translate(apostrophe_tbl)       # uniform apostrophe
    text = replace_by_tbl(text, shortform_tbl)  # split shortforms
    text = text.translate(remove_apostrophe_tbl)  # remove rest of '
    text = replace_by_tbl(text, restore_oclock)   # restore o'clock
    return text


def text2words(text, vocab_set=None):
    words = text_to_word_sequence(preprocess_text(text))
    words = [ word for word in words if word in word2idx]
    if not vocab_set is None:
        words = [ word for word in words if word in vocab_set]
    return words 

In [113]:
with open('processed_texts.csv', 'r', encoding='UTF-8') as file:
    texts = [line.strip('\n') for line in file]

# sequences of words
word_seqs = [text2words(text) for text in texts]

# flatten seqneces to one long sequence 
word_seq = [inner for outer in word_seqs for inner in outer]

# vocabulary - list of words that are used
vocab = build_vocab(word_seq, max_words)
vocab_size = len(vocab)
print(f'size of vocablary: {vocab_size}')

# remove all words that are not in the vocabulary
vocab_set = set(vocab)
word_seq = [word for word in word_seq if word in vocab_set]

# tokenized sequence of words
w2tk = {}
for i, word in enumerate(vocab):
    w2tk[word] = i

tkn_seq = [w2tk[word] for word in word_seq]
tkn_seq = np.array(tkn_seq)

def tk2emb(token):
    return glove[vocab[token]]

size of vocablary: 10000


In [114]:
# Sliding window to generate train data
seq = []
for i in range(len(tkn_seq)-sentence_len):
    seq.append(tkn_seq[i:i+sentence_len])

# reduce length of seq for performance reasons and of cause test_set
print('available samples: ', len(seq))
seq = seq[:max_samples]

# set data, label
X = []
y = []
for i in seq:
    X.append(i[:train_len])
    y.append(i[-1])

num_samples = len(y)

X = np.array(X)
y = np.array(y)

print('training samples: ', num_samples)

# split data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.4)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=.5)

available samples:  5070539
training samples:  3000000


In [115]:
# create the weights for the embedding layer

weights_matrix = np.zeros((vocab_size, embedding_dim))
words_found = 0

for i, word in enumerate(vocab):
    try: 
        weights_matrix[i] = glove[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(embedding_dim, ))

print(weights_matrix.shape)

(10000, 50)


In [116]:
def plot_hist(history):
    plt.figure(figsize=(12,4))
    # summarize history for accuracy
    plt.subplot(121)   
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])   
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    # summarize history for loss
    plt.subplot(122)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [117]:
# define model
model_2 = Sequential([
    Embedding(input_dim=vocab_size, 
              output_dim=embedding_dim, 
              input_length=train_len, 
              trainable=False, 
              weights=[weights_matrix]),
    LSTM(100, return_sequences=True),
    LSTM(100),
    Dense(100, activation='relu'),
    Dropout(0.1),
    Dense(vocab_size, activation='softmax')
])

model_2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [118]:
date_str = ''.join(['_'+str(t) for t in time.localtime(time.time())[:4]])
filepath = './models/model_2_weights' + date_str +'.hdf5'
checkpoint1 = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True)
checkpoint2 = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
callbacks_list = [checkpoint1, checkpoint2]

In [123]:
m2_history = model_2.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs = 20, batch_size = 1024, callbacks = callbacks_list)
plot_hist(m2_history)

Epoch 1/20
1758/1758 [==============================] - ETA: 0s - loss: 5.7079 - accuracy: 0.1134
Epoch 00001: val_loss improved from inf to 5.54575, saving model to ./models/model_2_weights_2020_7_23_21.hdf5
1758/1758 [==============================] - 831s 473ms/step - loss: 5.7079 - accuracy: 0.1134 - val_loss: 5.5457 - val_accuracy: 0.1277
Epoch 2/20
1758/1758 [==============================] - ETA: 0s - loss: 5.4722 - accuracy: 0.1310
Epoch 00002: val_loss improved from 5.54575 to 5.37527, saving model to ./models/model_2_weights_2020_7_23_21.hdf5
1758/1758 [==============================] - 838s 477ms/step - loss: 5.4722 - accuracy: 0.1310 - val_loss: 5.3753 - val_accuracy: 0.1386
Epoch 3/20
1758/1758 [==============================] - ETA: 0s - loss: 5.3348 - accuracy: 0.1401
Epoch 00003: val_loss improved from 5.37527 to 5.28239, saving model to ./models/model_2_weights_2020_7_23_21.hdf5
1758/1758 [==============================] - 839s 477ms/step - loss: 5.3348 - accuracy: 0.1

KeyboardInterrupt: 

In [130]:
def gen(model, sentence, max_len=30):
    ''' Generates a sequence given a string seq using specified model until the total sequence length
    reaches max_len'''
    # Tokenize the input string
    tokenized_sent = [w2tk[word] for word in text2words(sentence, vocab_set)]
    max_len = max_len+len(tokenized_sent)
    # If sentence is not as long as the desired sentence length, we need to 'pad sequence' so that
    # the array input shape is correct going into our LSTM. the `pad_sequences` function adds 
    # zeroes to the left side of our sequence until it becomes 19 long, the number of input features.
    while len(tokenized_sent) < max_len:
        padded_sentence = tf.keras.preprocessing.sequence.pad_sequences([tokenized_sent][-19:],maxlen=19)
        op = model.predict(np.asarray(padded_sentence).reshape(1,-1))
        tokenized_sent.append(op.argmax())
        
    return " ".join([vocab[tk] for tk in tokenized_sent]) + '\n'

In [131]:
sen1 = "Good evening everyone. My name"
sen2 = "Donald Trump"
sen3 = "Climate change"
sen4 = "My research about"

print(gen(model_2, sen1))
print(gen(model_2, sen2))
print(gen(model_2, sen3))
print(gen(model_2, sen4))

good evening everyone my name was a and i was a little bit of the and i was a little bit of the and i was a little bit of the and i was a

donald trump is the first thing is that i am going to show you a little bit of the world and i am going to show you a little bit of the

climate change is the same thing that is the same thing that is the same thing that is the same thing that is the same thing that is the same thing that

my research about the world is the same thing that is the same thing that is the same thing that is the same thing that is the same thing that is the same

